Design a dashboard layout that includes an interactive component to display sales, profits, and customer count based on a selected date range.A functional dashboard where users can filter by  date range and view dynamic visualizations.



Features of the Dashboard

Date Range Picker:

1.Users can select a specific date range to filter data dynamically.
2.The dcc.DatePickerRange component provides a convenient date range selection interface.

Dynamic Visualizations:

1.Sales Graph: Line chart showing sales trends over the selected date range.
2.Profit Graph: Bar chart displaying profits for the selected dates.
3.Customer Count Graph: Scatter plot illustrating customer counts.

Responsive Design:
1.The layout adjusts to fit the user's screen size.
2.Works across various devices (desktop, tablet, mobile).


In [6]:
import pandas as pd
import numpy as np

# Set a seed for reproducibility
np.random.seed(42)

# Create the dataset with non-linear data
data = {
    "Date": pd.date_range(start="2023-01-01", periods=100),
    "Sales": np.random.normal(loc=5000, scale=1500, size=100).cumsum(),  # Random walk for Sales
    "Profit": np.random.normal(loc=1000, scale=300, size=100).cumsum(),  # Random walk for Profit
    "Customers": np.random.randint(50, 500, size=100)  # Random customer count
}

# Convert data to DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv("sales_data.csv", index=False)

print("Dataset saved as sales_data.csv")


Dataset saved as sales_data.csv


In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Load the dataset from CSV file
df = pd.read_csv('sales_data.csv')

# Initialize the Dash app
app = dash.Dash(__name__)

# Layout
app.layout = html.Div([
    html.H1("Sales Dashboard", style={'textAlign': 'center'}),
    
    # Date Range Picker
    html.Div([
        html.Label("Select Date Range:"),
        dcc.DatePickerRange(
            id='date-range-picker',
            start_date=df['Date'].min(),
            end_date=df['Date'].max(),
            display_format="YYYY-MM-DD",
            style={'marginBottom': '20px'}
        )
    ]),

    # Graphs
    html.Div([
        dcc.Graph(id='sales-graph'),
        dcc.Graph(id='profit-graph'),
        dcc.Graph(id='customer-graph'),
    ])
])

# Callbacks to update graphs based on date range
@app.callback(
    [Output('sales-graph', 'figure'),
     Output('profit-graph', 'figure'),
     Output('customer-graph', 'figure')],
    [Input('date-range-picker', 'start_date'),
     Input('date-range-picker', 'end_date')]
)
def update_graphs(start_date, end_date):
    # Filter data based on selected date range
    filtered_data = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]

    # Sales Graph
    sales_fig = px.line(
        filtered_data, 
        x="Date", y="Sales",
        title="Sales Over Time",
        labels={'Sales': 'Sales Amount'}
    )

    # Profit Graph
    profit_fig = px.bar(
        filtered_data,
        x="Date", y="Profit",
        title="Profit Over Time",
        labels={'Profit': 'Profit Amount'},
        color_discrete_sequence=["green"]
    )

    # Customer Graph
    customer_fig = px.scatter(
        filtered_data,
        x="Date", y="Customers",
        title="Customer Count Over Time",
        labels={'Customers': 'Customer Count'},
        color_discrete_sequence=["blue"]
    )

    return sales_fig, profit_fig, customer_fig

# Run the app
if __name__ == '__main__':
     app.run_server(debug=True,port=8052)
